In [96]:
!nvidia-smi

Thu Mar 16 10:48:08 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   52C    P0    58W / 149W |  10913MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [111]:
# Load pickled data
import pickle
import numpy as np
import tensorflow as tf

# keras layers
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D


# keras utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

# keras existing models
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
tf.python.control_flow_ops = tf

In [112]:
# load data
with open('../data/traffic_signs/train.p', mode='rb') as f:
    data = pickle.load(f)

# preprocess data
X_train, y_train = data['features'], data['labels']
X_train = np.array(X_train / 255.0 - 0.5 )
y_train = to_categorical(y_train)
n_classes = len(y_train[0])

In [113]:
# load data
with open('../data/traffic_signs/valid.p', mode='rb') as f:
    data = pickle.load(f)

# preprocess data
X_cv, y_cv = data['features'], data['labels']
X_cv = np.array(X_cv / 255.0 - 0.5 )
y_cv = to_categorical(y_cv)

In [114]:
# load data
with open('../data/traffic_signs/test.p', mode='rb') as f:
    data = pickle.load(f)

# preprocess data
X_test, y_test = data['features'], data['labels']
X_test = np.array(X_test / 255.0 - 0.5 )
y_test = to_categorical(y_test)

In [115]:
# Initial Setup for Keras
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

In [116]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [117]:
epochs = 20
history = model.fit(X_train, y_train,
                    nb_epoch=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Train on 34799 samples, validate on 12630 samples
Epoch 1/20
34799/34799 [==============================] - 9s - loss: 0.9700 - acc: 0.7395 - val_loss: 0.6330 - val_acc: 0.8291
Epoch 2/20
34799/34799 [==============================] - 8s - loss: 0.2678 - acc: 0.9276 - val_loss: 0.5731 - val_acc: 0.8726
Epoch 3/20
34799/34799 [==============================] - 8s - loss: 0.1817 - acc: 0.9492 - val_loss: 0.5639 - val_acc: 0.8732
Epoch 4/20
34799/34799 [==============================] - 8s - loss: 0.1483 - acc: 0.9580 - val_loss: 0.5518 - val_acc: 0.8882
Epoch 5/20
34799/34799 [==============================] - 8s - loss: 0.1239 - acc: 0.9645 - val_loss: 0.5742 - val_acc: 0.8879
Epoch 6/20
34799/34799 [==============================] - 8s - loss: 0.1111 - acc: 0.9676 - val_loss: 0.5726 - val_acc: 0.8878
Epoch 7/20
34799/34799 [==============================] - 8s - loss: 0.0916 - acc: 0.9741 - val_loss: 0.5450 - val_acc: 0.8982
Epoch 8/20
34799/34799 [==============================] - 8s 

In [123]:
metrics = model.evaluate(X_test, y_test)
for m in range(len(model.metrics_names)):
    metric_name = model.metrics_names[m]
    metric_value = metrics[m]
    print('\n{}: {:.2f}'.format(metric_name, metric_value))

Testing
12512/12630 [============================>.] - ETA: 0s
loss: 0.76

acc: 0.89


In [ ]:
# data generator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest')

# data generator
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

# batches of augmented image data
train_generator = train_datagen.flow(
    X_train,
    target_size=(224, 224),
    batch_size=batch_size)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(
    X_cv,
    target_size=(224, 224),
    batch_size=batch_size)

# vgg model
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

# compile
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# fit augmented data
model.fit_generator(
    train_generator,
    samples_per_epoch=2000,
    nb_epoch=50,
    validation_data=validation_generator,
    nb_val_samples=800)
model.save_weights('first_try.h5')  # always save your weights after training or during training

# score
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])